## 完全ではないが，ある程度データは取れる
- 現状は日本基準で５つまでなら確認が取れた（年度末二件，第一四半期，第二四半期それぞれ一件ずつ）
## 問題点
- 企業ごとにタグの使い方が若干異なる（予期しないエラーがまだまだ発生しそう）
- 米国基準のものが一つだけ取得できたが，型式がひどい




In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import math
import csv

#選択行削除
import re
from xml.sax.saxutils import unescape

In [2]:
def normalize_XBRL(XBRL):
    with open(XBRL, encoding='utf-8') as f:
        html = f.read()
    # 「（」「）」はすべて半角に
    html = html.replace("（", "(")
    html = html.replace("）", ")")
    #tableタグだけ残す，それ以外のタグで囲まれていない部分を削除
    html  = re.sub("<table", "table\n<", html)
    html  = re.sub("</table", "tableend\n<", html)
    html  = re.sub("<br />", "", html)
    html  = re.sub("<td", "td<", html)
    html  = re.sub("</td>", "tdend\n", html)
    html  = re.sub("</tr>", "trend", html)
    
    text=""
    for i in html:
        text=text+i
        if i ==">":
            text=re.sub("<.*>", "", text)
            text=re.sub("{.*}", "", text)
            if text[-1:]!="\n":
                text = text+"\n"
    text1=text

    #テキスト処理
    text1=re.sub("&#160;", "None", text1)
    text1=re.sub("△\n", "-", text1)
    text1=re.sub("△", "-", text1)
    text1=re.sub(".root", "-", text1)
    text1=re.sub(",", "", text1)
    text1=re.sub("．", ".", text1)
    text1=re.sub("\ufeff\n", "", text1)
    text1=re.sub("－", "None", text1)
    text1=re.sub("：", "", text1)
    text1=re.sub(" ", "", text1)
    text1=re.sub("\n\n", "\n", text1)
    text1=re.sub("　", "", text1)
    text1=re.sub("０", "0", text1)
    text1=re.sub("１", "1", text1)
    text1=re.sub("２", "2", text1)
    text1=re.sub("３", "3", text1)
    text1=re.sub("４", "4", text1)
    text1=re.sub("５", "5", text1)
    text1=re.sub("６", "6", text1)
    text1=re.sub("７", "7", text1)
    text1=re.sub("８", "8", text1)
    text1=re.sub("９", "9", text1)
    

    return(text1)

In [3]:
def text_tag(text):
    #予測がいつまでなのかを取得(基本的に　３．　OO年度連結業績予測などになっているため，それを元に取得)
    pre_cnt=0
    pre = ""
    for i in range(len(text),-1,-1):
        if "連結業績予想(" in text[i-7:i]:
            pre_cnt=i
            break
    for i in range(pre_cnt,-1,-1):
        if "." in text[i-1:i]:
            pre = text[i:pre_cnt-1]
            break

    #tdで囲まれた部分の改行は削除して，tdを削除
    tx=""
    text1=""
    td_flag=False
    for i in text:
        #print(i)
        if "tdend" in text1:
            td_flag=False
        if "td" in text1:
            td_flag=True

        if td_flag:
            if i=="\n":
                i=""
        text1+=i
    text1=re.sub("tdend", "", text1)
    text1=re.sub("td", "\n", text1)
    text1=re.sub(r"\n+", "\n", text1)
    return(text1,pre)

In [4]:
def create_text_list(text):

    #tableタグで囲まれた部分をリストに入れる
    text_list = []
    table_cnt_first=0
    table_cnt_end=0
    non_text_list = []
    non_table_cnt_first=0
    append_text=""
    non_text=""
    for i in range(1,len(text)):
        if text[i-8:i] == "tableend":
        #tableタグに囲まれた文章をlistに追加
            table_cnt_end = i-8
            append_text=text[table_cnt_first:table_cnt_end]+"\n"
            append_text=re.sub("trend\ntrend","trend\n",append_text)
            append_text=re.sub("tableendtable","",append_text)
            append_text=re.sub(r'\n+', "\n",append_text)
            append_text=re.sub(r'\ntrend\nNone',"",append_text)
            #append_text=re.sub(r'\nNonetrend',"",append_text)
            text_list.append(append_text)
            table_cnt_first=0
            table_cnt_end=0
            #tableタグに囲まれている部分を追加したので囲まれていない部分のカウント初期化
            non_table_cnt_first=i

    #tableタグの始まりの判定    
        elif text[i-5:i] == "table":
            if table_cnt_first==0:
                table_cnt_first=i

            #tableタグに囲まれていない部分をlistに追加
            non_text=text[non_table_cnt_first:i-5]
            non_text_list.append(non_text)
    
    #listの中に（注）がある部分は削除する         
    pop_list=[]
    cnt=0
    for i in text_list:

        #if "(注)" in i :
          #  pop_list.append(cnt)
        if "(参考)" in i:
            pop_list.append(cnt)
        elif "(役職名)" in i:
            pop_list.append(cnt)
        cnt +=1

    for i in pop_list[::-1]:
        #print(i)
        text_list.pop(i)
    return(text_list,non_text_list)

In [5]:
def create_table_list(text_list,column_dic):
    new_li=[]
    n_cnt=0
    col=""
    column_li=[i for i in column_dic.keys()]
    #一番可能性が高い表をlistのトップに持ってくる
    for cnt,table in enumerate(text_list):
        n_cnt=0
        col=""
        for i,word in enumerate(table,1):
            if word=="\n":
                #columnになる可能性のある単語がcolumn辞書に存在する場合は高い確率で目的の表であると考えられる
                col = table[n_cnt:i-1]
                #col= re.sub("trend","",col)
                if col in column_li:
                    new_li.append(table)
                    break
                n_cnt=i
    return(new_li)

In [6]:
def create_list(table_list):
    number_list = []
    col_list=[]
    number = []
    col=""
    num_cnt=0
    col_cnt=0
    len_col=0
    add_col=0
    non_cnt=0
    for table in table_list:
        if "table" in table:
            table=table[table.find("table")+5:]
        for i in range(0,len(table)):
            if table[i:i+5] !="trend":
                if table[i:i+1] == "\n":
                    col=table[col_cnt+1:i]
                    add_col+=1
                    if col=="" and col_list!=[] and len_col==0:
                        len_col=len(col_list)
                    if col!="":
                        col_list.append(col)
                    col_cnt=i
                
            else:
                col=table[col_cnt+1:i]
                col_list.append(col)
                #追加したコラムが１つだけなら削除してやり直し
                if add_col==1:
                    add_col=0
                    col_list.pop(-1)
                    col_cnt=i+5
                    continue
                add_col=0
                num_cnt=i+5
                break
                
        #col_listの先頭がNoneでなければNoneを追加
        if (col_list!=[]) and col_list[non_cnt]!="None":
            #col_list.insert(len(column),"None")
            col_list.insert(non_cnt,"None")
            non_cnt=len(col_list)
            
        #column以降にある数字の部分を回す
        for i in range(num_cnt,len(table)+1):
            if table[i:i+1] == "\n":
                if table[i-5:i]=="trend":
                    if number!=[]:
                        number.append(table[num_cnt+1:i-5])
                        number_list.append(number)
                        number=[]
                        num_cnt=i

                elif table[i-5:i]!="trend":
                    number.append(table[num_cnt+1:i])
                    num_cnt=i
                    
    #[col_list.insert(k, col_list[k]) for k in range(len_col-1,-1,-1)]
    
    return(col_list,number_list)

In [7]:
def create_NoneAndCol(column,number):
    #columnをNoneを起点に分ける
    number_list=[]
    column_list=[]
    pop_list=[]
    cn=1
    for i in range(1,len(column)):
        if column[i] =="None":
            column_list.append(column[cn:i])
            cn=i+1
        elif i==len(column)-1:
            column_list.append(column[cn:i+1])
            
            
    #numberにNoneを入れる
    for cnt,num in enumerate(number):
        #if len(num) <3:
          #  pop_list.append(cnt)
        if ("円銭" in num) or ("百万円" in num):
            if "None" not in num:
                num.insert(0,"None")
        
        #後々Noneがあるとエラーになるのでいらないものは削除
        if num[0] != "None":
            num[0] = re.sub("None","",num[0])
        number_list.append(num)
        

    for i in pop_list[::-1]:
        #print(i)
        number_list.pop(i)
    return(column_list,number_list)

In [8]:
def create_index(number):
    index_list=[[]*i for i in range(0,len(number))]
    in_cnt=0
    index_list[in_cnt].append("None")

    for i in range(1,len(number)):
        if number[i][0]== "None":
            in_cnt+=1
            index_list[in_cnt].append("None")
        else:
            index_list[in_cnt].append(number[i][0])
    return(index_list)

In [9]:
def create_new_number_list(number,column_list):
    new_num=[]
    cn=0
    for cnt,num in enumerate(number):
        if num[0]== "None":
            if len(new_num)!=len(column_list):
                if len(column_list[cn])>2:

                    for i in range(0,len(num),2):
                        if i>len(column_list[cn])-1:
                            continue
                        if (len(num)-1) > (len(column_list[cn])):
                            column_list[cn].insert(i,column_list[cn][i])

                        else:
                            break
                new_num.append([column_list[cn][k]+num[k+1] for k in range(0,len(column_list[cn]))])
                new_num[cn].insert(0,"None")
                number[cnt]=new_num[cn]
                if cn!=len(column_list)-1:
                    cn+=1
    
    return(number)

In [126]:
def create_DataFrame(number,index_list):
    #最大でも４つまでの表しか作成しない．目的の表は２つだけなので
    new_df1 = pd.DataFrame
    new_df2 = pd.DataFrame
    new_df3 = pd.DataFrame
    new_df4=pd.DataFrame
    new_df = pd.DataFrame
    cn=1
    num_cnt=0
    for cnt,index in enumerate(index_list):

        if index!=[] and cn==1:
            new_df1=pd.DataFrame([number[k][1:] for k in range(num_cnt,len(index)+num_cnt)],index=index)
            num_cnt+=len(index)
            cn+=1
            continue
        elif index!=[] and cn==2:
            new_df2=pd.DataFrame([number[k][1:] for k in range(num_cnt,len(index)+num_cnt)],index=index)
            num_cnt+=len(index)
            #インデックスが同じなら合わせる
            if index==index_list[cnt-1]:
                new_df=pd.concat([new_df1,new_df2],axis=1)
            else:
                cn+=1

        elif index!=[] and cn==3:
            new_df3=pd.DataFrame([number[k][1:] for k in range(num_cnt,len(index)+num_cnt)],index=index)
            num_cnt+=len(index)
            cn+=1
        elif index!=[] and cn==4:
            new_df4=pd.DataFrame([number[k][1:] for k in range(num_cnt,len(index)+num_cnt)],index=index)
            break
    #dfがNoneだったらdf1を出力
    if str(type(new_df))=="<class 'type'>": 
        return(new_df1,new_df2,new_df3,new_df4)
          
    else:
        return(new_df,new_df2,new_df3,new_df4)

In [11]:
def reset_column(df1,df2):
    sh1=df1.shape
    sh2 = df2.shape
    #column1をリセット
    df1.columns = range(sh1[1])
    #column2をリセット
    df2.columns = range(sh2[1])
    
    return(df1,df2)

In [148]:
def reset_column1(df1):
    sh1=df1.shape

    #column1をリセット
    df1.columns = range(sh1[1])

    return(df1)

In [72]:
def select_df(df2,df3,df4,pre_term):
    
    try:
        if ("通期" in str(df2.index)) or( "予想" in str(df2.index ))or("累計" in str(df2.index)):
            return(df2)
        elif ("通期" in str(df3.index)) or( "予想" in str(df3.index ))or("累計" in str(df3.index)): 
            return(df3)
        elif ("通期" in str(df4.index)) or( "予想" in str(df4.index ))or("累計" in str(df4.index)):
            return(df4)
        
        else:
            index_list=[]
            index_list = [i for i in df2.index]
            for j in index_list:
                if pre_term[:4] in j:
                    return(df2)
                pass
            
            index_list = [i for i in df3.index]
            for j in index_list:
                if pre_term[:4] in j:
                    return(df3)
                pass
            
            index_list = [i for i in df4.index]
            for j in index_list:
                if pre_term[:4] in j:
                    return(df4)
                pass
    except:
        pass

In [68]:
def select_df2(df,df2,df3,df4):
    if str(type(df))=="<class 'NoneType'>":
        try:
            if "累計" in str(df2.index) or "累計" in str(df2.index):
                return(df2)
            elif "累計" in str(df3.index) or "累計" in str(df3.index):
                return(df3)
            elif "累計" in str(df4.index) or "累計" in str(df4.index):
                return(df4)
        except:
            pass

In [205]:
def create_df1_column(df1,column_list):
    df1_col=[ i for i in df1.columns]
    cnt_0 = df1_col.count(0)
    new_col=[]
    cnt=1
    end=0
    try:
        for zero_cnt in range(0,cnt_0):
            for i in range(cnt,len(df1_col)):
                if df1_col[i]==0:
                    #columnとdfの長さが同じ時（表１）
                    if len(column_list[zero_cnt]) == i:
                        new_col= column_list[0]
                    #columnとdfの長さが同じ時（表２）
                    elif (zero_cnt!=0) and (len(column_list[zero_cnt]) == i-end+1):
                        new_col= column_list[0]
                    else:
                        cnt=i+1
                        break
                #エラーが起きないように最後になったらforを抜ける        
                elif i==len(df1_col)-1:
                    cnt=i+3
                    break
            #抜けた後，column数が同じならそのまま加える
            if (len(column_list[zero_cnt]) == i-end+1):
                for k in column_list[zero_cnt]:
                    new_col.append(k)
            else:
                #0の手前までのcolumnを作成
                cn=0
                for j in range(end,cnt-1):
                    if j %2==0:
                        new_col.append(column_list[zero_cnt][cn])
                    else:
                        new_col.append(column_list[zero_cnt][cn])
                        cn+=1
                end=j+2
        df1.loc["index"]=new_col
    except:
        for k in range(0,3):
            if len(new_col)!=len(df1_col):
                new_col.append("None")
            else:
                break
        df1.loc["index"]=new_col
    return(df1,zero_cnt+1)

In [15]:
def create_df_column(df1,column_list,zero_cnt):
    try:
        df_col=[ i for i in df.columns]
        new_col=[]
        cn=0
        for i in range(zero_cnt,len(column_list)):
            new_col=[]
            cn=0
            for j in range(0,len(df_col)):
                if cn < len(column_list[i]):
                    #print(j)
                    if j %2==0:
                        new_col.append(column_list[i][cn])
                        #print(column_list[i][cn])
                    else:
                        new_col.append(column_list[i][cn])
                        #print(column_list[i][cn])
                        cn+=1

            if  len(df_col)==len(new_col):
                df.loc["index"]=new_col
                break
        return(df1)
    except:
        return(df1)

In [16]:
def check_text(text_list,non_text):
    new_text_list=[]
    new_text=""

    for txt in text_list:
        if txt.count("。")>=1:
            new_text+=txt
    new_text_list.append(new_text)
    
    new_text=""
    for txt in non_text:
        if txt.count("。")>=1:
            new_text+=txt
    new_text_list.append(new_text)
    
    return(new_text_list)

In [17]:
def NoPredict_or(NoPredict_text):
    cnt=0
    cnt=0
    for i in NoPredict_text:
        if ("未定"in i)or ("控え" in i) or ("困難" in i)or("M&A" in i) or("コロナウイルス" in i) or("合理的に算出" in i)or("非連結" in i)or("IFRS" in i)or("精査中" in i):
            cnt+=1
        elif ("速やかに開示" in i) or ("開示する予定"in i)or ("ＩＦＲＳ" in i):
            cnt+=1
    if cnt>0:
        return(True)
    else:
        return(False)

In [18]:
def tuki_reset(number):
    for cnt,li in enumerate(number):
        if "通期" in li:
            if li[0]!="通期":
                number[cnt].remove("通期")
                number[cnt+1].insert(0,"通期")
    return(number)

In [19]:
def trend_reset(number):
    for cnt,li in enumerate(number):
        for cnt2,li2 in enumerate(li):
            if "trend" in li2:
                number[cnt].pop(cnt2)
                break
    return(number)

In [263]:
def col_reset(column):
    for cnt,li in enumerate(column):
        pop_list=[]
        for cnt2,li2 in enumerate(li):
            if "※" in li2:
                pop_list.append(cnt2)
        for j in pop_list[::-1]:
            column[cnt].pop(j)
    return(column)

In [157]:
def main(file): 
    col_dic={
    "売上高":"Sales",
    "営業利益":"Operating",
    "事業利益":"Business",
    "経常利益":"Ordinary",
    "親会社株主に帰属する当期純利益":"ParentNetIncome",
    "親会社株主に帰属する四半期純利益":"QuarterParentNetIncome",
    "1株当たり当期純利益":"OneNetIncome",
    "1株当たり四半期純利益":"QuarterOneNetIncome"

    }
    li=[]
    NoPreFlag=False
    errFlag=False
    try:
        text=normalize_XBRL(file)
        text,pre_term = text_tag(text)
        text_list,non_text=create_text_list(text)
        NoPredict_text=check_text(text_list,non_text)
        table_list=create_table_list(text_list,col_dic)
        column,number=create_list(table_list)
        column_list,number = create_NoneAndCol(column,number)
        number=tuki_reset(number)
        number=trend_reset(number)
        index_list=create_index(number)
        df1,df2,df3,df4=create_DataFrame(number,index_list)
        df=select_df(df2,df3,df4,pre_term)
        df1,zero_cnt,=create_df1_column(df1,column_list)
        df=create_df_column(df,column_list,zero_cnt)
        df1=reset_column1(df1)
        if str(type(df))=="<class 'NoneType'>":
            if NoPredict_or(NoPredict_text):
                    NoPreFlag=True
        else:
            df=reset_column1(df)
    except:
        errFlag=True
        if str(type(df))=="<class 'NoneType'>":
            if NoPredict_or(NoPredict_text):
                NoPreFlag =True
                    
    return(df1,df,pre_term,NoPredict_text,NoPreFlag,errFlag,table_list)

In [21]:
#自作関数読み込み
import pre_process.xbrl_process3 as xb

In [156]:
main(li[16])[1]

In [181]:
NoPredict_or(NoPredict_text)

True

In [191]:
"ebrl_df/27020/20200511408747.htm"

In [164]:
import pickle
out='new2/err/non_list2.pkl'
with open(out, 'rb') as p:
    out_list=pickle.load(p)

In [22]:
col_dic={
    "売上高":"Sales",
    "営業利益":"Operating",
    "事業利益":"Business",
    "経常利益":"Ordinary",
    "親会社株主に帰属する当期純利益":"ParentNetIncome",
    "親会社株主に帰属する四半期純利益":"QuarterParentNetIncome",
    "1株当たり当期純利益":"OneNetIncome",
    "1株当たり四半期純利益":"QuarterOneNetIncome"
}

In [281]:
i_list[34:]
out_list[i_list[37]]

'xbrl_data/73390/20211109428229.htm'

In [275]:
i = i_list[37]
text=normalize_XBRL(out_list[i])
text,pre_term = text_tag(text)
text_list,non_text=create_text_list(text)
NoPredict_text=check_text(text_list,non_text)
table_list=create_table_list(text_list,col_dic)
column,number=create_list(table_list)
column_list,number = create_NoneAndCol(column,number)
number=tuki_reset(number)
number=trend_reset(number)
index_list=create_index(number)
df1,df2,df3,df4=create_DataFrame(number,index_list)
df=select_df(df2,df3,df4,pre_term)
df1,zero_cnt,=create_df1_column(df1,column_list)
df=create_df_column(df,column_list,zero_cnt)
df1=reset_column1(df1)
if str(type(df))=="<class 'NoneType'>":
    if NoPredict_or(NoPredict_text):
            NoPreFlag=True
else:
    df=reset_column1(df)

In [280]:
table_list

['\nNone\n経常収益\n経常利益\n親会社株主に帰属する中間純利益trend\nNone\n百万円\n％\n百万円\n％\n百万円\n％trend\n2022年3月期中間期\n13702\nNone\n179\nNone\n100\nNonetrend\n2021年3月期中間期\nNone\nNone\nNone\nNone\nNone\nNonetrend\n',
 '\nNone\n経常収益\n経常利益\n親会社株主に帰属する中間純利益trend\nNone\n百万円\n％\n百万円\n％\n百万円\n％trend\n2022年3月期中間期\n13702\nNone\n129\nNone\n64\nNonetrend\n2021年3月期中間期\nNone\nNone\nNone\nNone\nNone\nNonetrend\n',
 '\nNone\n経常収益\n経常利益\n親会社株主に帰属する当期純利益\n1株当たり当期純利益trend\nNone\n百万円\n％\n百万円\n％\n百万円\n％\n円銭trend\n通期\n28600\n25.0\n0\nNone\n-50\nNone\n-4.63trend\n']

In [279]:
df

,0,1,2,3,4,5,6
None,百万円,％,百万円,％,百万円,％,円銭
通期,28600,25.0,0,None,-50,None,-4.63
index,経常収益,経常収益,経常利益,経常利益,親会社株主に帰属する当期純利益,親会社株主に帰属する当期純利益,1株当たり当期純利益


In [268]:
table_list

['\nNone\n売上高\nSaaSARR※1\nEBITDA※2\n営業利益\n経常利益trend\nNone\n百万円\n％\n百万円\n％\n百万円\n％\n百万円\n％\n百万円\n％trend\n2022年11月期第1四半期\n4755\n37.2\n12904\n39.9\n-1159\nNone\n-1638\nNone\n-1655\nNonetrend\n2021年11月期第1四半期\n3466\n44.8\n9224\n35.4\n358\nNone\n80\nNone\n74\nNonetrend\n',
 '\nNone\n親会社株主に帰属する四半期純利益\n1株当たり四半期純利益\n潜在株式調整後1株当たり四半期純利益trend\nNone\n百万円\n％\n円銭\n円銭trend\n2022年11月期第1四半期\n-1668\nNone\n-31.28\nNonetrend\n2021年11月期第1四半期\n64\nNone\n1.35\n1.30trend\n',
 '\nNone\n売上高\nSaaSARR※1\nEBITDA※2trend\nNone\n百万円\n％\n百万円\n％\n百万円\n％trend\n通期\n20322～21886\n30.0～40.0\n15718～16841\n40.0～50.0\nNone\nNonetrend\n',
 '\nNone\n営業利益\n経常利益\n親会社株主に帰属する当期純利益\n1株当たり当期純利益trend\nNone\n百万円\n％\n百万円\n％\n百万円\n％\n円銭trend\n通期\nNone\nNone\nNone\nNone\nNone\nNone\nNonetrend\n',
 '\nNone\n売上高\nSaaSARR※1\nEBITDA※2trend\nNone\n百万円\n％\n百万円\n％\n百万円\n％trend\n第2四半期\n4752～5072\n19.0～27.0\n13576～13982\n34.0～38.0\n-1900～-1400\nNonetrend\n',
 '\nNone\n営業利益\n経常利益\n親会社株主に帰属する当期純利益\n1株当たり当期純利益trend\nNone\n百万円\n％\n百万円\n％\n百万円\n％\n円銭tren

In [75]:
if str(type(df))=="<class 'NoneType'>":
    if "累計" in df2.index[0] or "累計" in df2.index[1]:
        print("a")

In [521]:
df=select_df2(df,df2,df3,df4)

In [204]:
df1

,0,1,2,3,4,5,6,7,0,1,2,3,4,5,6
None,百万円,％,百万円,％,百万円,％,百万円,％,円銭,円銭,％,％,％,百万円,％
2022年3月期,55255,3.6,1119,-56.2,1126,-60.8,451,-72.9,23.28,None,1.6,2.0,2.0,5318,-17.2
2021年3月期,53341,-4.1,2554,-23.5,2874,-17.0,1664,-22.8,85.32,None,6.2,5.2,4.8,6425,-10.0
index,売上高,売上高,営業利益,営業利益,経常利益,経常利益,親会社株主に帰属する当期純利益,親会社株主に帰属する当期純利益,1株当たり当期純利益,潜在株式調整後1株当たり当期純利益,自己資本当期純利益率,総資産経常利益率,売上高営業利益率,EBITDA,None


In [120]:
index_list

[['None', '2019年3月期', '2018年3月期'],
 ['None', '2019年3月期', '2018年3月期'],
 ['None', '2019年3月期'],
 ['None', '2019年3月期'],
 ['None', '第2四半期(累計)', '通期'],
 ['None', '通期'],
 ['None', '2019年3月期', '2018年3月期'],
 ['None', '2019年3月期', '2018年3月期'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [347]:
#column,number=create_list(table_list)
column_list,number = create_NoneAndCol(column,number)
index_list=create_index(number)
df1,df2,df3,df4=create_DataFrame(number,index_list)
df=select_df(df2,df3,df4,pre_term)
df1,zero_cnt,=create_df1_column(df1,column_list)
df=create_df_column(df,column_list,zero_cnt)
df1,df=reset_column(df1,df)

In [25]:
li=[]
for i in range(0,len(out_list)):
    flag=main(out_list[i])[4]
    if flag:
        li.append(out_list[i])

In [26]:
li

['xbrl_data/18480/20210201453300.htm',
 'xbrl_data/22160/20180801490075.htm',
 'xbrl_data/22160/20180508430012.htm',
 'xbrl_data/28010/20210421497149.htm',
 'xbrl_data/36040/20180207466096.htm',
 'xbrl_data/40630/20220425526894.htm',
 'xbrl_data/40630/20210428402264.htm',
 'xbrl_data/40630/20200427400836.htm',
 'xbrl_data/40630/20190426412585.htm',
 'xbrl_data/40630/20180427422921.htm',
 'xbrl_data/49110/20180207466746.htm',
 'xbrl_data/51080/20200217466058.htm',
 'xbrl_data/51220/20191029413963.htm',
 'xbrl_data/68710/20220511538979.htm',
 'xbrl_data/68710/20220210586412.htm',
 'xbrl_data/68710/20210513415553.htm',
 'xbrl_data/68710/20210212462529.htm',
 'xbrl_data/68710/20200915492967.htm',
 'xbrl_data/68710/20200514413408.htm',
 'xbrl_data/68710/20200213463328.htm',
 'xbrl_data/68710/20191114426665.htm',
 'xbrl_data/68710/20190206470842.htm',
 'xbrl_data/68710/20181114437360.htm',
 'xbrl_data/68710/20180207465932.htm',
 'xbrl_data/68710/20171114417960.htm',
 'xbrl_data/67300/2019080

In [212]:
out_list

['xbrl_data/18480/20210201453300.htm',
 'xbrl_data/22160/20180801490075.htm',
 'xbrl_data/22160/20180508430012.htm',
 'xbrl_data/28010/20210421497149.htm',
 'xbrl_data/36040/20180207466096.htm',
 'xbrl_data/36070/20210701461195.htm',
 'xbrl_data/78930/20200130454295.htm',
 'xbrl_data/78930/20191030415993.htm',
 'xbrl_data/78930/20190730478786.htm',
 'xbrl_data/78930/20190130466080.htm',
 'xbrl_data/78930/20181030425823.htm',
 'xbrl_data/78930/20180730487967.htm',
 'xbrl_data/78930/20180129458323.htm',
 'xbrl_data/78930/20171030402627.htm',
 'xbrl_data/78930/20170728441732.htm',
 'xbrl_data/23760/20211101422369.htm',
 'xbrl_data/23760/20210806480628.htm',
 'xbrl_data/23760/20201105416227.htm',
 'xbrl_data/23760/20200806476815.htm',
 'xbrl_data/40630/20220425526894.htm',
 'xbrl_data/40630/20210428402264.htm',
 'xbrl_data/40630/20200427400836.htm',
 'xbrl_data/40630/20190426412585.htm',
 'xbrl_data/40630/20180427422921.htm',
 'xbrl_data/34070/20190510420533.htm',
 'xbrl_data/49110/2018020

In [112]:
df_col=[ i for i in df.columns]
new_col=[]
cn=0
for i in range(zero_cnt,len(column_list)):
    new_col=[]
    cn=0
    for j in range(0,len(df_col)):
        if cn < len(column_list[i]):
            print(j)
            if j %2==0:
                new_col.append(column_list[i][cn])
                print(column_list[i][cn])
            else:
                new_col.append(column_list[i][cn])
                print(column_list[i][cn])
                cn+=1

    if  len(df_col)==len(new_col):
        df.loc["index"]=new_col
        break

0
1株当たり四半期純利益
1
1株当たり四半期純利益
2
潜在株式調整後1株当たり四半期純利益
3
潜在株式調整後1株当たり四半期純利益
0
売上高
1
売上高
2
営業利益
3
営業利益
4
経常利益
5
経常利益
6
親会社株主に帰属する当期純利益
7
親会社株主に帰属する当期純利益
8
1株当たり当期純利益


In [113]:
new_col

['売上高',
 '売上高',
 '営業利益',
 '営業利益',
 '経常利益',
 '経常利益',
 '親会社株主に帰属する当期純利益',
 '親会社株主に帰属する当期純利益',
 '1株当たり当期純利益']

In [528]:
df

,0,1,2,3,4,5,6,7,8
None,百万円,％,百万円,％,百万円,％,百万円,％,円銭
第2四半期(累計),106400,27.9,34100,46.0,33800,46.2,24200,45.8,671.10


In [177]:
import tqdm
li=[]
i_list=[]
for i in (range(0,len(out_list))):
    try:
        text=normalize_XBRL(out_list[i])
        text,pre_term = text_tag(text)
        text_list,non_text=create_text_list(text)
        NoPredict_text=check_text(text_list,non_text)
        table_list=create_table_list(text_list,col_dic)
        column,number=create_list(table_list)
        column_list,number = create_NoneAndCol(column,number)
        number=tuki_reset(number)
        number=trend_reset(number)
        index_list=create_index(number)
        df1,df2,df3,df4=create_DataFrame(number,index_list)
        df=select_df(df2,df3,df4,pre_term)
        #df=select_df2(df,df2,df3,df4)
        df1,zero_cnt,=create_df1_column(df1,column_list)
        df=create_df_column(df,column_list,zero_cnt)
        df1,df=reset_column(df1,df)
        i_list.append(i)
    except:
        if str(type(df))=="<class 'NoneType'>":
            if not NoPredict_or(NoPredict_text):
                li.append(out_list[i])
                print(NoPredict_text)
                print("-------------------------------------",i)
        else:
            i_list.append(i)

['\n当社は、「企業は社会の公器」として、短期的な利益ではなく、長期的な視点に立った経営を優先する社会の実現を目指す、そのような企業群の一翼を担うことが重要な経営目標であると認識しています。したがって、社員・顧客・仕入先・地域社会・地球といったすべての社中に貢献することにより企業価値を上げ、その結果として株主にも利益をもたらすという考え方である「公益資本主義」の理念に賛同しています。None四半期決算の開示あるいは業績予想の開示については、当該諸制度・諸規則(慣習)のありかたそのものによって、目先の株価変動や短期業績への過剰な傾注、投資意思決定における投機的な視野への偏狭を誘発し、企業活動における長期的な視点に立った研究開発投資、設備投資および人材育成を妨げることのほか、企業の「社会の公器」としての役割、すなわち、社員、顧客、取引先、地域社会に対する還元など社会貢献を疎かにさせる懸念が生じています。Noneこのような懸念を解消するべく、「未来投資戦略2017」(内閣府)において提言された、決算短信における業績予想の様式廃止は2017年2月実現しました。None以上の観点から、当社グループは、2018年3月期まで通期の連結業績予想を開示していましたが、2017年2月実現した日本政府の制度改正の意義、趣旨に賛同し、いち早くこれを実践することで、2019年3月期以降について業績予想の開示を行っていません。trend\n', 'None(注)「収益認識に関する会計基準」(企業会計基準第29号2020年3月31日。以下「収益認識会計基準」という。)等を当連結会計年度の期首から適用しており、2022年3月期に係る各数値は、当該会計基準等を適用した後の数値となっています。None(2)連結財政状態None(注)「収益認識に関する会計基準」(企業会計基準第29号2020年3月31日。以下「収益認識会計基準」という。)等を当連結会計年度の期首から適用しており、2022年3月期に係る各数値は、当該会計基準等を適用した後の数値となっています。None(2)連結財政状態table\nNone\n総資産\n純資産\n自己資本比率\n1株当たり純資産trend\nNone\n百万円\n百万円\n％\n円銭trend\n2022年3月期\n8708\n729\n8.4\n31.29trend\

['', '平成31年3月期の期末配当につきましては、平成31年3月期第3四半期決算発表時に、年度業績見通し等を踏まえて判断し、公表する予定です。trend(注)詳細は、9ページ「2.四半期連結財務諸表及び主な注記(3)四半期連結財務諸表に関する注記事項(会計方針の変更・会計上の見積りの変更・修正再表示)」をご覧ください。(4)発行済株式数(普通株式)(注)詳細は、9ページ「2.四半期連結財務諸表及び主な注記(3)四半期連結財務諸表に関する注記事項(会計方針の変更・会計上の見積りの変更・修正再表示)」をご覧ください。(4)発行済株式数(普通株式)table\n①期末発行済株式数(自己株式を含む)\n31年3月期2Q\n950321402株\n30年3月期\n950321402株trend\n②期末自己株式数\n31年3月期2Q\n67721153株\n30年3月期\n67710915株trend\n③期中平均株式数(四半期累計)\n31年3月期2Q\n882605722株\n30年3月期2Q\n882639519株trend※四半期決算短信は公認会計士又は監査法人の四半期レビューの対象外です※業績予想の適切な利用に関する説明、その他特記事項(将来予測情報の適切な利用に関する説明)本資料に記載されている将来に関する記述には、本資料の発表日現在の将来に関する前提・見通し・計画に基づく予測が含まれております。実際の業績は、今後様々な要因によって大きく異なる結果となる可能性があります。trend']
------------------------------------- 10
['', '(1)当四半期連結累計期間における重要な子会社の異動(連結範囲の変更を伴う特定子会社の異動)無(2)四半期連結財務諸表の作成に特有の会計処理の適用有(注)詳細は、9ページ「2.四半期連結財務諸表及び主な注記(3)四半期連結財務諸表に関する注記事項(四半期連結財務諸表の作成に特有の会計処理の適用)」をご覧ください。(3)会計方針の変更・会計上の見積りの変更・修正再表示(1)当四半期連結累計期間における重要な子会社の異動(連結範囲の変更を伴う特定子会社の異動)無(2)四半期連結財務諸表の作成に特有の会計処理の適用有(注)詳細は、9ページ「2.四半期連結財務諸表及び主な注記(3)四半期連結財

['', 'None3.平成31年12月期の連結業績予想(平成31年1月1日～平成31年12月31日)当社及び当社グループにおける次期の業績見通しにつきましては、店舗のリブランド推進や、デリバリー事業の複合宅配事業拠点の出店計画の推進状況等、様々な要因により変動する事が予想されます。現在、当該推進計画を織り込んだ中期計画を鋭意策定中でありますので、平成31年12月期の連結業績予想につきましては、当該計画の策定を完了次第、公表させて頂きます。NoneNone注記事項None3.平成31年12月期の連結業績予想(平成31年1月1日～平成31年12月31日)当社及び当社グループにおける次期の業績見通しにつきましては、店舗のリブランド推進や、デリバリー事業の複合宅配事業拠点の出店計画の推進状況等、様々な要因により変動する事が予想されます。現在、当該推進計画を織り込んだ中期計画を鋭意策定中でありますので、平成31年12月期の連結業績予想につきましては、当該計画の策定を完了次第、公表させて頂きます。NoneNone注記事項table\ntrend\n(1)期中における重要な子会社の異動(連結範囲の変更を伴う特定子会社の異動)\n無\nNone\nNonetrend']
------------------------------------- 26
['', 'None3.平成31年12月期の連結業績予想(平成31年1月1日～平成31年12月31日)当社及び当社グループにおける次期の業績見通しにつきましては、店舗のリブランド推進や、デリバリー事業の複合宅配事業拠点の出店計画の推進状況等、様々な要因により変動する事が予想されます。現在、当該推進計画を織り込んだ中期計画を鋭意策定中でありますので、平成31年12月期の連結業績予想につきましては、当該計画の策定を完了次第、公表させて頂きます。NoneNone注記事項None3.平成31年12月期の連結業績予想(平成31年1月1日～平成31年12月31日)当社及び当社グループにおける次期の業績見通しにつきましては、店舗のリブランド推進や、デリバリー事業の複合宅配事業拠点の出店計画の推進状況等、様々な要因により変動する事が予想されます。現在、当該推進計画を織り込んだ中期計画を鋭意策定中でありますので、平成31年12月期の連結業績予想につき

['\n(注)None「自己資本比率」は、(期末純資産の部合計None期末新株予約権None期末非支配株主持分)を期末資産の部合計で除して算出しております。trend\n\n2019年3月期の「親会社株主に帰属する当期純利益」は9500億円を目標としております(2018年5月15日公表の目標値から変更しております)。(当社グループは、銀行業、信託銀行業、証券業、クレジットカード・貸金業等の金融サービス業を展開しておりますが、これらの業務には、経済情勢、相場環境等に起因するさまざまな不確実性が存在するため、業績予想に代えて、親会社株主に帰属する当期純利益の目標値を記載しております。)trend\n\n本資料には、当社又は当社グループの業績、財政状態その他経営全般に関する予想、見通し、目標、計画等の将来に関する記述が含まれています。かかる記述は、現時点における予測、認識、評価等を基礎として記載されています。また、将来の予想、見通し、目標、計画等を策定するためには、一定の前提(仮定)を使用しています。これらの記述ないし前提(仮定)は、その性質上、将来その通りに実現するという保証はなく、客観的には不正確であったり、実際の結果と大きく乖離する可能性があります。そのような事態の原因となりうる不確実性やリスクの要因は多数あります。その内、現時点において想定しうる主な事項については、決算短信、有価証券報告書、ディスクロージャー誌、AnnualReportをはじめとした当社の公表済みの各種資料の最新のものをご参照ください。trend\n', 'Nonetable\n(注)None「自己資本比率」は、(期末純資産の部合計None期末新株予約権None期末非支配株主持分)を期末資産の部合計で除して算出しております。trendNonetable\n2019年3月期の「親会社株主に帰属する当期純利益」は9500億円を目標としております(2018年5月15日公表の目標値から変更しております)。(当社グループは、銀行業、信託銀行業、証券業、クレジットカード・貸金業等の金融サービス業を展開しておりますが、これらの業務には、経済情勢、相場環境等に起因するさまざまな不確実性が存在するため、業績予想に代えて、親会社株主に帰属する当期純利益の目標値を記載しております。)trendNone(注)詳細は、【添

['\n(注)当社では、連結損益計算書において事業収益として記載しているものを売上高と読み替えて表示しております。trend\n\n2022年12月期の業績予想につきましては、現時点で合理的な業績予想の算定ができないことから、記載しておりません。なお、当該理由等は、添付資料P.6「1.経営成績等の概況(5)今後の見通し」をご覧ください。trend\n\n(注)当社では、損益計算書において事業収益として記載しているものを売上高と読み替えて表示しております。trend\n\n※\n決算短信は公認会計士又は監査法人の監査の対象外です。trend\n\n・本資料に記載されている業績見通し等の将来に関する記述は、当社が現在入手している情報及び合理的であると判断する一定の前提に基づいており、実際の業績等は様々な要因により大きく異なる可能性があります。業績予想の前提となる条件及び業績予想のご利用にあたっての注意事項等については、添付資料P.6「1.経営成績等の概況(5)今後の見通し」をご覧ください。trend\n', 'Nonetable\n(注)当社では、連結損益計算書において事業収益として記載しているものを売上高と読み替えて表示しております。trendNonetable\n2022年12月期の業績予想につきましては、現時点で合理的な業績予想の算定ができないことから、記載しておりません。なお、当該理由等は、添付資料P.6「1.経営成績等の概況(5)今後の見通し」をご覧ください。trendNonetable\n(注)当社では、損益計算書において事業収益として記載しているものを売上高と読み替えて表示しております。trendNonetable\n※\n決算短信は公認会計士又は監査法人の監査の対象外です。trendNonetable\n・本資料に記載されている業績見通し等の将来に関する記述は、当社が現在入手している情報及び合理的であると判断する一定の前提に基づいており、実際の業績等は様々な要因により大きく異なる可能性があります。業績予想の前提となる条件及び業績予想のご利用にあたっての注意事項等については、添付資料P.6「1.経営成績等の概況(5)今後の見通し」をご覧ください。trend']
------------------------------------- 54
['\n(注)

['\n3.平成30年12月期の連結業績予想(平成30年1月1日～平成30年12月31日)平成30年12月期の連結業績予想につきましては開示を見合わせております。詳細は、添付資料「1.当四半期決算に関する定性的情報(3)連結業績予想などの将来予測情報に関する説明」をご参照ください。(注)直近に公表されている業績予想からの修正の有無無trend\n', 'table\n3.平成30年12月期の連結業績予想(平成30年1月1日～平成30年12月31日)平成30年12月期の連結業績予想につきましては開示を見合わせております。詳細は、添付資料「1.当四半期決算に関する定性的情報(3)連結業績予想などの将来予測情報に関する説明」をご参照ください。(注)直近に公表されている業績予想からの修正の有無無trend※四半期決算短信は公認会計士又は監査法人の四半期レビューの対象外です※業績予想の適切な利用に関する説明、その他特記事項(将来に関する記述等についてのご注意)本資料に記載されている業績見通し等の将来に関する記述は、当社が現在入手している情報及び合理的であると判断する一定の前提に基づいており、その達成を当社として約束する趣旨のものではありません。また、実際の業績等は様々な要因により大きく異なる可能性があります。業績予想の前提となる条件及び業績予想のご利用にあたっての注意事項等につきましては、添付資料「1.当四半期決算に関する定性的情報(3)連結業績予想などの将来予測情報に関する説明」をご覧ください。trend']
------------------------------------- 74
['\n3.平成30年12月期の連結業績予想(平成30年1月1日～平成30年12月31日)平成30年12月期の連結業績予想につきましては開示を見合わせております。詳細は、添付資料「今後の見通し」をご参照ください。trend\n\n※決算短信は監査の対象外です※業績予想の適切な利用に関する説明、その他特記事項(将来に関する記述等についてのご注意)本資料に記載されている業績見通し等の将来に関する記述は、当社が現在入手している情報及び合理的であると判断する一定の前提に基づいており、その達成を当社として約束する趣旨のものではありません。また、実際の業績等は様々な要因により大きく異なる可能性があり

['\n※決算短信は公認会計士又は監査法人の監査の対象外です※業績予想の適切な利用に関する説明、その他特記事項(将来に関する記述等についてのご注意)本資料に記載されている業績見通し等の将来に関する記述は、当社が現在入手している情報及び合理的であると判断する一定の前提に基づいており、実際の業績等は様々な要因により大きく異なる可能性があります。(決算補足説明資料の入手方法について)当社は、2019年5月13日に機関投資家・アナリスト向け決算説明会を開催する予定です。この説明会で配布する決算説明会資料については開催後速やかに当社ホームページに掲載する予定です。trend\n', '(注)みずほ信託銀行株式会社(以下、「ESOP信託口」という。)が所有する当社株式429000株(議決権の数4290個)につきましては、上記期末自己株式数に含めております。trendtable\n※決算短信は公認会計士又は監査法人の監査の対象外です※業績予想の適切な利用に関する説明、その他特記事項(将来に関する記述等についてのご注意)本資料に記載されている業績見通し等の将来に関する記述は、当社が現在入手している情報及び合理的であると判断する一定の前提に基づいており、実際の業績等は様々な要因により大きく異なる可能性があります。(決算補足説明資料の入手方法について)当社は、2019年5月13日に機関投資家・アナリスト向け決算説明会を開催する予定です。この説明会で配布する決算説明会資料については開催後速やかに当社ホームページに掲載する予定です。trend']
------------------------------------- 89
['\n※決算短信は公認会計士又は監査法人の監査の対象外です※業績予想の適切な利用に関する説明、その他特記事項(将来に関する記述等についてのご注意)本資料に記載されている業績見通し等の将来に関する記述は、当社が現在入手している情報及び合理的であると判断する一定の前提に基いており、実際の業績等は様々な要因により大きく異なる可能性があります。(決算補足説明資料の入手方法について)当社は、平成30年5月10日に機関投資家・アナリスト向け決算説明会を開催する予定です。この説明会で配布する決算説明会資料については開催後速やかに当社ホームページでに掲載する予定です。trend\

['', '(注)2022年4月12日付プレスリリース「ＭＢＯの実施及び応募の推奨に関するお知らせ」において公表いたしましたとおり、株式会社BCJ-60(以下「公開買付者」といいます。)による当社の発行済普通株式(以下「当社株式」といいます。)及び新株予約権に対する公開買付け並びにその後の一連の手続きにより、当社株式が上場廃止となる予定であることから2023年2月期の配当予想は記載しておりません。None3.2023年2月期の連結業績予想(2022年3月1日～2023年2月28日)2022年4月12日付プレスリリース「ＭＢＯの実施及び応募の推奨に関するお知らせ」において公表いたしましたとおり、公開買付者による公開買付け及びその後の一連の手続きにより、当社株式が上場廃止となる予定であることから2023年2月期の連結業績予想は記載しておりません。None※注記事項(注)2022年4月12日付プレスリリース「ＭＢＯの実施及び応募の推奨に関するお知らせ」において公表いたしましたとおり、株式会社BCJ-60(以下「公開買付者」といいます。)による当社の発行済普通株式(以下「当社株式」といいます。)及び新株予約権に対する公開買付け並びにその後の一連の手続きにより、当社株式が上場廃止となる予定であることから2023年2月期の配当予想は記載しておりません。None3.2023年2月期の連結業績予想(2022年3月1日～2023年2月28日)2022年4月12日付プレスリリース「ＭＢＯの実施及び応募の推奨に関するお知らせ」において公表いたしましたとおり、公開買付者による公開買付け及びその後の一連の手続きにより、当社株式が上場廃止となる予定であることから2023年2月期の連結業績予想は記載しておりません。None※注記事項table\n(1)期中における重要な子会社の異動(連結範囲の変更を伴う特定子会社の異動)\n無\nNone\nNonetrend(注)特定子会社の異動には該当いたしませんが、当社の連結子会社でありましたPT.MerdisInternationalの全株式を譲渡したため、連結の範囲から除外しております。None(2)会計方針の変更・会計上の見積りの変更・修正再表示(注)特定子会社の異動には該当いたしませんが、当社の連結子会社でありましたPT.MerdisInterna

['\n2021年3月期通期につきましては、売上高は前期比▲15～20％減少する見込みです。詳細は添付資料Ｐ.5「1.経営成績の概況(4)今後の見通し」をご覧ください。trend\n\n(将来事象に関する記述等についてのご注意)本資料に記載されている業績見通し等の将来に関する記述は、当社が現在入手している情報及び合理的であると判断する一定の前提に基づいており、その達成を当社として約束する趣旨のものではありません。また、実際の業績は様々な要因により大きく異なる可能性があります。None(決算説明会内容の入手方法)当社は2020年6月30日にアナリスト向け説明会を開催し、その模様を同日又は翌日に当社ホームページへ掲載する予定です。説明会の資料は2020年6月30日にTDnet及び当社ホームページに掲載します。trend\n', 'Nonetable\n2021年3月期通期につきましては、売上高は前期比▲15～20％減少する見込みです。詳細は添付資料Ｐ.5「1.経営成績の概況(4)今後の見通し」をご覧ください。trendNonetable\n(将来事象に関する記述等についてのご注意)本資料に記載されている業績見通し等の将来に関する記述は、当社が現在入手している情報及び合理的であると判断する一定の前提に基づいており、その達成を当社として約束する趣旨のものではありません。また、実際の業績は様々な要因により大きく異なる可能性があります。None(決算説明会内容の入手方法)当社は2020年6月30日にアナリスト向け説明会を開催し、その模様を同日又は翌日に当社ホームページへ掲載する予定です。説明会の資料は2020年6月30日にTDnet及び当社ホームページに掲載します。trend']
------------------------------------- 118
['\n2021年3月期通期につきましては、売上高は前期比▲15～20％減少する見込みです。詳細は添付資料Ｐ.5「1.経営成績の概況(4)今後の見通し」をご覧ください。trend\n\n(将来事象に関する記述等についてのご注意)本資料に記載されている業績見通し等の将来に関する記述は、当社が現在入手している情報及び合理的であると判断する一定の前提に基づいており、その達成を当社として約束する趣旨のものではありません。また、

['\n※決算短信は公認会計士又は監査法人の監査の対象外です※業績予想の適切な利用に関する説明、その他特記事項(将来に関する記述等についてのご注意)本資料に記載されている業績見通し等の将来に関する記述は、当社が現在入手している情報及び合理的であると判断する一定の前提に基づいており、その達成を当社として約束する趣旨のものではありません。また、実際の業績等は様々な要因により大きく異なる可能性があります。trend\n', '(注)潜在株式調整後1株当たり当期純利益については、潜在株式が存在しないため記載しておりません。(2)連結財政状態(注)潜在株式調整後1株当たり当期純利益については、潜在株式が存在しないため記載しておりません。(2)連結財政状態table\n総資産\n純資産\n自己資本比率\n1株当たり純資産trend\n百万円\n百万円\n％\n円銭trend\ntrend\n2022年3月期\n6580\n2487\n37.8\n227.12trend\n2021年3月期\n6591\n2262\n34.3\n206.60trend(注)当社は定款において中間期末及び期末日を配当基準日と定めております。尚、2023年3月期につきましては、利益環境が更に厳しくなると予想されるため、年間配当金は0円とする予定です。trendtable\n※決算短信は公認会計士又は監査法人の監査の対象外です※業績予想の適切な利用に関する説明、その他特記事項(将来に関する記述等についてのご注意)本資料に記載されている業績見通し等の将来に関する記述は、当社が現在入手している情報及び合理的であると判断する一定の前提に基づいており、その達成を当社として約束する趣旨のものではありません。また、実際の業績等は様々な要因により大きく異なる可能性があります。trend']
------------------------------------- 140
['\n(注)1.2021年12月期の潜在株式調整後1株当たり当期純利益については、潜在株式は存在するものの、1株当たり当期純損失であるため、記載しておりません。2.2021年12月期より連結財務諸表を作成しているため、2020年12月期の数値及び対前期増減率については記載しておりません。また、自己資本当期純利益率及び総資産経常利益率は連結初年度の

In [166]:
len(li)

108

In [167]:
len(out_list)

146

In [168]:
dic={}
for i in range(0,len(li)):
     dic[li[i][10:15]]=1

In [169]:
len(dic.keys())

36

In [170]:
dic={}
for i in range(0,len(out_list)):
     dic[out_list[i][10:15]]=1

In [171]:
len(dic.keys())

45

In [193]:
for i in i_list:
    print(out_list[i])

xbrl_data/34070/20190510420533.htm
xbrl_data/59330/20220428531754.htm
xbrl_data/59330/20220131576772.htm
xbrl_data/59330/20211019413502.htm
xbrl_data/59330/20210720469325.htm
xbrl_data/59330/20210426499613.htm
xbrl_data/59330/20210129451596.htm
xbrl_data/83030/20170928480493.htm
xbrl_data/23890/20190206471708.htm
xbrl_data/23890/20190207472510.htm
xbrl_data/23890/20190207472614.htm
xbrl_data/23890/20181108433037.htm
xbrl_data/23890/20180802491018.htm
xbrl_data/23890/20180509431497.htm
xbrl_data/36880/20220512544654.htm
xbrl_data/36880/20220214589496.htm
xbrl_data/36880/20211110430409.htm
xbrl_data/36880/20210805480217.htm
xbrl_data/36880/20210512413855.htm
xbrl_data/36880/20210210461638.htm
xbrl_data/36880/20201109419020.htm
xbrl_data/36880/20200811479865.htm
xbrl_data/36880/20200513412147.htm
xbrl_data/36880/20200212462298.htm
xbrl_data/36880/20191112425078.htm
xbrl_data/36880/20190805482587.htm
xbrl_data/36880/20190514426188.htm
xbrl_data/94680/20220512543039.htm
xbrl_data/94680/2022

In [179]:
out_list[0]

'xbrl_data/34070/20190510420533.htm'

In [132]:
li[16:22]

['xbrl_data/79450/20220510537341.htm',
 'xbrl_data/81070/20171108412761.htm',
 'xbrl_data/81540/20180508429702.htm',
 'xbrl_data/99730/20190801481160.htm',
 'xbrl_data/99730/20190328496993.htm',
 'xbrl_data/99730/20190214477860.htm']

In [131]:
li

['xbrl_data/53430/20220510537296.htm',
 'xbrl_data/53430/20210324483627.htm',
 'xbrl_data/53430/20200416494750.htm',
 'xbrl_data/53430/20190507415202.htm',
 'xbrl_data/53430/20190123462529.htm',
 'xbrl_data/53430/20181002413621.htm',
 'xbrl_data/53430/20180726485386.htm',
 'xbrl_data/53430/20180302481598.htm',
 'xbrl_data/54010/20190205470139.htm',
 'xbrl_data/54010/20181030425598.htm',
 'xbrl_data/54010/20180706477808.htm',
 'xbrl_data/61430/20171114417968.htm',
 'xbrl_data/61430/20170809453320.htm',
 'xbrl_data/67150/20190614455003.htm',
 'xbrl_data/67150/20190419407282.htm',
 'xbrl_data/72010/20171106410086.htm',
 'xbrl_data/79450/20220510537341.htm',
 'xbrl_data/81070/20171108412761.htm',
 'xbrl_data/81540/20180508429702.htm',
 'xbrl_data/99730/20190801481160.htm',
 'xbrl_data/99730/20190328496993.htm',
 'xbrl_data/99730/20190214477860.htm',
 'xbrl_data/78650/20210301471772.htm',
 'xbrl_data/78650/20200214465867.htm',
 'xbrl_data/78650/20181218451312.htm',
 'xbrl_data/78650/2018022

In [ ]:
78650
48000
37930
30420
67400
39030

In [ ]:
34070
59330
83030
23890
36880
94680
31330
39940
29820
73390

In [282]:
pre_dic={
    78650:1,
    48000:1,
    37930:1,
    30420:1,
    67400:1,
    39030:1,
    34070:1,
    59330:1,
    83030:1,
    23890:1,
    36880:1,
    94680:1,
    31330:1,
    39940:1,
    29820:1,
    73390:1
}

In [292]:
len(pre_list)

66

In [290]:
pre_list=[]
for i in range(0,len(out_list)):
    if int(out_list[i][10:15]) in pre_dic.keys():
        pre_list.append(out_list[i])

# 連結予想があるかの確認

In [112]:
text=normalize_XBRL(non_list[0])
text,pre_term = text_tag(text)
text_list,non_text=create_text_list(text)

In [114]:
check_text(text_list,non_text)

['\n(注3)上記「配当の状況」は、普通株式に係る配当の状況です。甲種類株式(非上場)の配当の状況については、別紙の「甲種類株式の配当の状況」をご覧ください。(注4)当社は2019年6月25日に開催された定時株主総会で、「定款一部変更の件」が承認されたことを受けて、2019年度より決算期を3月31日から12月31日に変更しました。従いまして、経過期間となる2019年12月期は、2019年4月1日から2019年12月31日までの9か月決算となります。trend\n\n上記に記載した予想数値は、現時点で入手可能な情報に基づき判断した見通しであり、多分に不確定な要素を含んでおります。実際の業績は、石油・天然ガス価格、生産・販売計画、プロジェクト開発スケジュール、政府規制、財務・税制条件等の変化により、上記予想数値と異なる場合があります。なお、上記予想に関する事項は添付資料4ページを参照してください。trend\n',
 'NoneNonetable\n(注3)上記「配当の状況」は、普通株式に係る配当の状況です。甲種類株式(非上場)の配当の状況については、別紙の「甲種類株式の配当の状況」をご覧ください。(注4)当社は2019年6月25日に開催された定時株主総会で、「定款一部変更の件」が承認されたことを受けて、2019年度より決算期を3月31日から12月31日に変更しました。従いまして、経過期間となる2019年12月期は、2019年4月1日から2019年12月31日までの9か月決算となります。trendNonetable\n当社は、2019年6月25日に開催された定時株主総会で、「定款一部変更の件」が承認されたことを受けて、2019年度より決算期を3月31日から12月31日に変更しました。従いまして、経過期間となる2019年12月期は、当社および仮決算子会社※等につきましては、2019年4月1日から2019年12月31日の9か月間を連結対象期間、12月決算の子会社につきましては、従来通り、2019年1月1日から2019年12月31日の12か月間を連結対象期間としています。※仮決算子会社とは決算日は12月31日であるものの、連結決算に与える影響が相対的に大きいため、3月31日に組み替えて決算を行った子会社であります。None(参考)下記の％表示(調整後増減率)は、当社および仮

In [38]:
new_text_list=[]
for i in range(0,len(non_list)):
    text=normalize_XBRL(non_list[i])
    text,pre_term = text_tag(text)
    text_list,non_text=create_text_list(text)
    new_text=""
    
    for txt in text_list:
        if txt.count("。")>1:
            new_text+=txt
    new_text_list.append(new_text)

In [46]:
cnt=0
for i in new_text_list:
    if ("未定"in i)or ("控える" in i) or ("困難" in i)or("M&A" in i):
        print(i)
        cnt+=1


3.2021年3月期の連結業績予想(2020年4月1日～2021年3月31日)2021年3月期の連結業績予想につきましては、新型コロナウイルスの感染拡大による影響を検証中であり、現時点では合理的に算定することが困難であることから未定としています。今後、業績予想の算定が可能となった時点で速やかに公表いたします。trend

※決算短信は公認会計士又は監査法人の監査の対象外です※業績予想の適切な利用に関する説明、その他特記事項1.2021年3月期の連結業績予想につきましては、新型コロナウイルスの感染拡大による影響を検証中であり、現時点では合理的に算定することが困難であることから未定としています。今後、業績予想の算定が可能となった時点で速やかに公表いたします。2.当社は、2020年5月15日(金)にアナリスト・機関投資家向け説明会を開催する予定です。この説明会で使用する資料につきましては、当社ホNoneムページに掲載いたします。trend


3.2021年3月期の連結業績予想(2020年4月1日～2021年3月31日)新型コロナウイルス感染症の世界的な拡大により、国内外ともに売上への影響が想定されます。感染防止のための外出規制や自粛、経済活動の再開時期など、各国の状況が未だ不透明であり、今期の業績予想を立案することが困難な状況です。業績予想については、予測可能となった時点で公表いたします。trend


3.2021年3月期の連結業績予想(2020年4月1日～2021年3月31日)新型コロナウィルス感染症の世界的な拡大により、当社グループにおいて各地域で影響が生じております。今後も、主に国内食料品製造・販売事業、海外食料品製造・販売事業、海外食料品卸売事業においては、業務用向け商品の出荷に影響を受けることが想定されますが、業績予想算定について未確定な要素が多く、数値を示すことが困難な状況です。業績予想については、合理的に予測可能となった時点で公表いたします。trend

※決算短信は公認会計士又は監査法人の監査の対象外です※業績予想の適切な利用に関する説明、その他特記事項(将来に関する記述等についてのご注意)新型コロナウィルス感染症の世界的な拡大により、当社グループにおいて各地域で影響が生じております。今後も、主に国内食料品製造・販売事業、海外食料品製造・販売事業、海

In [47]:
cnt

179

In [255]:
new_df=pd.DataFrame

In [44]:
"None" in str(df.index)

True

## 以下notebook

In [36]:
with open('test04.htm', encoding='utf-8') as f:
    html = f.read()
html  = re.sub(">", ">\n\n", html)
print(html)

<?xml version="1.0" encoding="utf-8"?>


<html xmlns="http://www.w3.org/1999/xhtml" xmlns:xbrldi="http://xbrl.org/2006/xbrldi" xmlns:xbrli="http://www.xbrl.org/2003/instance" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:link="http://www.xbrl.org/2003/linkbase" xmlns:num="http://www.xbrl.org/dtr/type/numeric" xmlns:nonnum="http://www.xbrl.org/dtr/type/non-numeric" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:tse-acedjpsm-84100="http://www.xbrl.tdnet.info/jp/tse/tdnet/ac/edjp/sm/84100/20220506384100" xmlns:tse-ed-t="http://www.xbrl.tdnet.info/taxonomy/jp/tse/tdnet/ed/t/2014-01-12" xmlns:iso4217="http://www.xbrl.org/2003/iso4217" xmlns:ix="http://www.xbrl.org/2008/inlineXBRL" xmlns:ixt="http://www.xbrl.org/inlineXBRL/transformation/2011-07-31" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">

<head>

<title>

00_9663395503403.htm</title>

<meta http-equiv="X-UA-Compatible" content="IE=Edge" />

<meta http-equiv="content-type" content="text/html; charset=utf-8" />

<style t

In [76]:
#non_list

In [80]:
df = pd.read_csv('xbrl_df/13010/20170619410034_1.csv',encoding="utf-8")

In [51]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import math
import csv
import json
import pickle
import tqdm
#選択行削除
import re
from xml.sax.saxutils import unescape
from base64 import b64decode

import zipfile
import os
import itertools

from collections import OrderedDict
import traceback
from lxml.etree import fromstring as etree_fromstring

In [55]:
company = pd.read_csv("TDNet_full_data/company_list.csv",encoding="shift-jis",index_col=0)
#上場企業リスト取得
#証券コードのlistを作成
com_list=[]
for i in range(0,len(company)):
    if company["上場区分"][i]=="上場":
        com_list.append(company["証券コード"][i])
    #if len(com_list)==3882:
        #print(i)
#最初に書いてある，証券コードが記入されていない企業が最後に存在するため削除
com_list = com_list[:-1]

In [65]:
non_list=[]
df2_none_list=[]
#for i in (range(0,10)):
for i in tqdm.tqdm(range(0,len(com_list))):
    
    #indexを開く
    json_open= open('TDNet_full_data/API_index/{}.json'.format(int(com_list[i])), 'r')
    js = json.load(json_open)
   # print(js)
    #break
    #使用するコードのllistを作成，合致するコードが一つでもあれば取得
    pdf_code_list = []
    for j in range(0,len(js["publiclyList"])):
        for k in range(0,len(js["publiclyList"][j]["disclosureItems"])):
            if (js["publiclyList"][j]["disclosureItems"][k][2:] == "301" or 
                js["publiclyList"][j]["disclosureItems"][k][2:] == "304" or 
                js["publiclyList"][j]["disclosureItems"][k][2:] == "307" or
                js["publiclyList"][j]["disclosureItems"][k][2:] == "310"):
                if js["publiclyList"][j]["xbrlFlag"]=="1" and js["publiclyList"][j]["pdfSummaryFlag"]=="1":
                    pdf_code_list.append(js["publiclyList"][j]["disclosureNumber"])
                    break

100%|█████████████████████████████████████| 3881/3881 [00:01<00:00, 2568.12it/s]


In [66]:
pdf_code_list

[]

In [72]:
non_list=[]
df2_none_list=[]
for i in (range(0,1)):
#for i in tqdm.tqdm(range(0,len(com_list))):
    
    #indexを開く
    json_open= open('TDNet_full_data/API_index/{}.json'.format(int(com_list[i])), 'r')
    js = json.load(json_open)
   # print(js)
    #break
    #使用するコードのllistを作成，合致するコードが一つでもあれば取得
    pdf_code_list = []
    for j in range(0,len(js["publiclyList"])):
        for k in range(0,len(js["publiclyList"][j]["disclosureItems"])):
            if (js["publiclyList"][j]["disclosureItems"][k][2:] == "301" or 
                    js["publiclyList"][j]["disclosureItems"][k][2:] == "304" or 
                    js["publiclyList"][j]["disclosureItems"][k][2:] == "307" or
                    js["publiclyList"][j]["disclosureItems"][k][2:] == "310"):
                if js["publiclyList"][j]["xbrlFlag"]=="1" and js["publiclyList"][j]["pdfSummaryFlag"]=="1":
                    print(js["publiclyList"][j]["disclosureItems"])
                    pdf_code_list.append(js["publiclyList"][j]["disclosureNumber"])
                    break

['11310', '11350']
['11108', '11307']
['11304']
['11108', '11301']
['11310']
['11108', '11307']
['11304']
['11108', '11301']
['11310', '11350']
['11108', '11307']
['11304']
['11108', '11301']
['11310']
['11108', '11307']
['11304']
['11108', '11301', '11351']
['11310', '11350']
['11108', '11307']
['11304']
['11301']


In [73]:
pdf_code_list

['20220328512394',
 '20211224560318',
 '20210928403538',
 '20210708463658',
 '20210331488354',
 '20201224438982',
 '20201001498500',
 '20200713461156',
 '20200327485606',
 '20191219439108',
 '20190919499244',
 '20190624460038',
 '20190307487203',
 '20181205445580',
 '20180905403871',
 '20180613463090',
 '20180301480984',
 '20171201428656',
 '20170901466541',
 '20170615407199']

In [ ]:
"err_data2/27020/20200511408747.htm"

In [329]:
text=normalize_XBRL("err_data2/27020/20200511408747.htm")
text,pre_term = text_tag(text)
text_list,non_text=create_text_list(text)
table_list=create_table_list(text_list,col_dic)
column,number=create_list(table_list)
column_list,number = create_NoneAndCol(column,number)
index_list=create_index(number)
df1,df2,df3,df4=create_DataFrame(number,index_list)
df=select_df(df2,df3,df4,pre_term)
df1=create_df1_column(df1,column_list)
df=create_df_column(df,column)

In [292]:
column_list[-1]

['売上高', '営業利益', '経常利益', '親会社株主に帰属する当期純利益', '1株当たり当期純利益']

In [293]:
def create_df_column(df1,column_list):
    df1_col=[ i for i in df1.columns]
    new_col=[]
    cn=0
    for j in range(0,len(df1_col)):
        if j %2==0:
            new_col.append(column_list[-1][cn])
        else:
            new_col.append(column_list[-1][cn])
            cn+=1
    print(new_col)
    df1.loc["index"]=new_col
    return(df1)

In [294]:
df

,0,1,2,3,4,5,6,7,8
None,百万円,％,百万円,％,百万円,％,百万円,％,円銭
通期,287000,1.9,29000,3.5,28500,3.7,18200,7.8,136.88


In [295]:
create_df_column(df,column_list)

['売上高', '売上高', '営業利益', '営業利益', '経常利益', '経常利益', '親会社株主に帰属する当期純利益', '親会社株主に帰属する当期純利益', '1株当たり当期純利益']


,0,1,2,3,4,5,6,7,8
None,百万円,％,百万円,％,百万円,％,百万円,％,円銭
通期,287000,1.9,29000,3.5,28500,3.7,18200,7.8,136.88
index,売上高,売上高,営業利益,営業利益,経常利益,経常利益,親会社株主に帰属する当期純利益,親会社株主に帰属する当期純利益,1株当たり当期純利益


In [201]:
df1.index

Index([('None',), ('2020年12月期第1四半期',), ('2019年12月期第1四半期',), 'column'], dtype='object')

In [254]:
df1_col=[ i for i in df1.columns]
cnt_0 = df1_col.count(0)
new_col=[]
cnt=1
end=0
for zero_cnt in range(0,cnt_0):
    for i in range(cnt,len(df1_col)):
        if df1_col[i]==0:
            #columnとdfの長さが同じ時（表１）
            if len(column_list[zero_cnt]) == i:
                new_col= column_list[0]
            #columnとdfの長さが同じ時（表２）
            elif (zero_cnt!=0) and (len(column_list[zero_cnt]) == i-end+1):
                new_col= column_list[0]
            else:
                cnt=i+1
                break
        #エラーが起きないように最後になったらforを抜ける        
        elif i==len(df1_col)-1:
            cnt=i+3
            break
    #抜けた後，column数が同じならそのまま加える
    if (len(column_list[zero_cnt]) == i-end+1):
        for k in column_list[zero_cnt]:
            new_col.append(k)
    else:
        #0の手前までのcolumnを作成
        cn=0
        for j in range(end,cnt-1):
            if j %2==0:
                new_col.append(column_list[zero_cnt][cn])
            else:
                new_col.append(column_list[zero_cnt][cn])
                cn+=1
        end=j+2
kari_df = pd.DataFrame([new_col],index=["column"])
sh1=df1.shape
#column1をリセット
df1.columns = range(sh1[1])
new_df = pd.concat([kari_df,df1])

In [243]:
[a[0] for a in df1.index]

['None', '2020年12月期第1四半期', '2019年12月期第1四半期']

In [255]:
pd.concat([kari_df,df1])

,0,1,2,3,4,5,6,7,8,9
column,売上高,売上高,営業利益,営業利益,経常利益,経常利益,親会社株主に帰属する四半期純利益,親会社株主に帰属する四半期純利益,1株当たり四半期純利益,潜在株式調整後1株当たり四半期純利益
None,百万円,％,百万円,％,百万円,％,百万円,％,円銭,円銭
2020年12月期第1四半期,72286,4.9,7708,-8.2,7426,-14.3,4713,-14.8,35.45,None
2019年12月期第1四半期,68884,2.7,8396,-5.6,8665,-5.2,5535,-28.1,41.63,None


In [241]:
df1

,0,1,2,3,4,5,6,7,8,9
None,百万円,％,百万円,％,百万円,％,百万円,％,円銭,円銭
2020年12月期第1四半期,72286,4.9,7708,-8.2,7426,-14.3,4713,-14.8,35.45,None
2019年12月期第1四半期,68884,2.7,8396,-5.6,8665,-5.2,5535,-28.1,41.63,None


In [18]:
import tqdm
li=[]
for i in tqdm.tqdm(range(0,len(non_list))):
    try:
        text=normalize_XBRL(non_list[i])
        text,pre_term = text_tag(text)
        text_list,non_text=create_text_list(text)
        table_list=create_table_list(text_list,col_dic)
        column,number=create_list(table_list)
        column_list,number = create_NoneAndCol(column,number)
        index_list=create_index(number)
        df1,df2,df3,df4=create_DataFrame(number,index_list)
        df=select_df(df2,df3,df4,pre_term)
        df1=create_df1_column(df1,column_list)
        df=create_df_column(df,column_list)
    except:
        li.append(non_list[i])

100%|███████████████████████████████████████| 1154/1154 [00:32<00:00, 35.50it/s]


In [20]:
len(li)

439

In [21]:
len(non_list)

1154

In [22]:
dic={}
for i in range(0,len(li)):
     dic[li[i][10:15]]=1

In [23]:
len(dic.keys())

125

In [24]:
dic

{'18020': 1,
 '28010': 1,
 '29050': 1,
 '35280': 1,
 '79220': 1,
 '40420': 1,
 '41880': 1,
 '42150': 1,
 '41830': 1,
 '45440': 1,
 '49670': 1,
 '51050': 1,
 '42240': 1,
 '53430': 1,
 '54010': 1,
 '57420': 1,
 '59330': 1,
 '63610': 1,
 '63790': 1,
 '63170': 1,
 '64610': 1,
 '67150': 1,
 '68350': 1,
 '68610': 1,
 '69420': 1,
 '69410': 1,
 '72010': 1,
 '72050': 1,
 '72350': 1,
 '79560': 1,
 '77430': 1,
 '81010': 1,
 '80520': 1,
 '81070': 1,
 '80350': 1,
 '33500': 1,
 '81900': 1,
 '81650': 1,
 '98610': 1,
 '98760': 1,
 '99460': 1,
 '99360': 1,
 '22940': 1,
 '30320': 1,
 '30570': 1,
 '30630': 1,
 '83030': 1,
 '83040': 1,
 '83460': 1,
 '83310': 1,
 '83370': 1,
 '83590': 1,
 '83620': 1,
 '83690': 1,
 '83670': 1,
 '83060': 1,
 '83090': 1,
 '84110': 1,
 '84180': 1,
 '83540': 1,
 '85510': 1,
 '85620': 1,
 '71840': 1,
 '85300': 1,
 '84210': 1,
 '86090': 1,
 '86160': 1,
 '86240': 1,
 '37720': 1,
 '87660': 1,
 '88770': 1,
 '89250': 1,
 '32450': 1,
 '93180': 1,
 '94240': 1,
 '95010': 1,
 '95330': 1,

In [27]:
dic["30570"]

1

In [28]:
dic["31680"]

1

In [29]:
dic["31820"]

1

In [30]:
dic["32710"]

1

In [31]:
dic["34170"]

1

In [34]:
dic["40420"]

1

In [35]:
dic["43850"]

1

In [36]:
dic["47220"]

1

In [37]:
dic["48000"]

1

In [39]:
dic["49670"]

1

In [41]:
dic["65510"]

1

In [43]:
dic["70500"]

1

In [45]:
dic["77430"]

1

In [47]:
dic["81650"]

1

In [50]:
print(dic["83090"])
print(dic["85930"])
dic["99460"]

1
1


1

In [322]:
li[2]

'xbrl_data/49670/20201014403631.htm'

In [323]:
text=normalize_XBRL(li[2])
text,pre_term = text_tag(text)
text_list,non_text=create_text_list(text)
NoPredict_text=check_text(text_list,non_text)
table_list=create_table_list(text_list,col_dic)
column,number=create_list(table_list)
column_list,number = create_NoneAndCol(column,number)
index_list=create_index(number)
df1,df2,df3,df4=create_DataFrame(number,index_list)
df=select_df(df2,df3,df4,pre_term)
df1=create_df1_column(df1,column_list)
df=create_df_column(df,column_list)

ValueError: cannot set a row with mismatched columns

In [326]:
NoPredict_text

['',
 '(注)1.2020年12月期第1四半期より、「収益認識に関する会計基準」(企業会計基準第29号2018年3月30日)及び「収益認識に関する会計基準の適用指針」(企業会計基準適用指針第30号2018年3月30日)を早期適用しております。なお、原則的な取扱いに従って、新たな会計方針を過去の期間のすべてに遡及適用しております。これにより、2019年12月期第3四半期の対前年同四半期増減率は、当該会計方針の変更に伴い遡及修正を行ったため、記載しておりません。2.2020年12月期第1四半期より、当社及び国内連結子会社は、すべての有形固定資産について減価償却の方法を定額法に変更しております。また、一部の有形固定資産及び無形固定資産の耐用年数を実態にあわせた耐用年数に変更しております。従来の方法によった場合の金額(％表示は対前年同四半期増減率)を以下に記載しております。2020年12月期第3四半期(百万円)売上高営業利益経常利益親会社株主に帰属する四半期純利益105049-5.5％202562.1％217861.6％157653.4％(2)連結財政状態(注)1.2020年12月期第1四半期より、「収益認識に関する会計基準」(企業会計基準第29号2018年3月30日)及び「収益認識に関する会計基準の適用指針」(企業会計基準適用指針第30号2018年3月30日)を早期適用しております。なお、原則的な取扱いに従って、新たな会計方針を過去の期間のすべてに遡及適用しております。これにより、2019年12月期第3四半期の対前年同四半期増減率は、当該会計方針の変更に伴い遡及修正を行ったため、記載しておりません。2.2020年12月期第1四半期より、当社及び国内連結子会社は、すべての有形固定資産について減価償却の方法を定額法に変更しております。また、一部の有形固定資産及び無形固定資産の耐用年数を実態にあわせた耐用年数に変更しております。従来の方法によった場合の金額(％表示は対前年同四半期増減率)を以下に記載しております。2020年12月期第3四半期(百万円)売上高営業利益経常利益親会社株主に帰属する四半期純利益105049-5.5％202562.1％217861.6％157653.4％(2)連結財政状態table\n総資産\n純資産\n自己資本比率\n1株当たり純資産trend\

In [319]:
df1

,0,1,2,3,4,5,6,7
None,百万円,％,百万円,％,百万円,％,百万円,％
2020年12月期第3四半期,105049,-5.5,19515,-1.6,21042,-1.9,15258,0.1
2019年12月期第3四半期,111202,―,19831,―,21447,―,15245,―


In [320]:
df

,0,1,2,3,4,5,6,7,8
None,百万円,％,百万円,％,百万円,％,百万円,％,円銭
通期,154000,-2.7,25900,0.9,27600,-0.9,19200,0.3,245.64


In [321]:
column

['売上高',
 '営業利益',
 '経常利益',
 '親会社株主に帰属する四半期純利益',
 '2020年12月期第3四半期',
 '195.21',
 '売上高',
 '営業利益',
 '経常利益',
 '親会社株主に帰属する当期純利益',
 '1株当たり当期純利益']

In [287]:
df1_col=[ i for i in df1.columns]
cnt_0 = df1_col.count(0)
new_col=[]
cnt=1
end=0
try:
    for zero_cnt in range(0,cnt_0):
        for i in range(cnt,len(df1_col)):
            if df1_col[i]==0:
                #columnとdfの長さが同じ時（表１）
                if len(column_list[zero_cnt]) == i:
                    new_col= column_list[0]
                #columnとdfの長さが同じ時（表２）
                elif (zero_cnt!=0) and (len(column_list[zero_cnt]) == i-end+1):
                    new_col= column_list[0]
                else:
                    cnt=i+1
                    break
            #エラーが起きないように最後になったらforを抜ける        
            elif i==len(df1_col)-1:
                cnt=i+3
                break
        #抜けた後，column数が同じならそのまま加える
        if (len(column_list[zero_cnt]) == i-end+1):
            for k in column_list[zero_cnt]:
                new_col.append(k)
        else:
            #0の手前までのcolumnを作成
            cn=0
            for j in range(end,cnt-1):
                if j %2==0:
                    new_col.append(column_list[zero_cnt][cn])
                else:
                    new_col.append(column_list[zero_cnt][cn])
                    cn+=1
            end=j+2
    df1.loc["index"]=new_col
except:
    df1.loc["index"]=new_col

In [288]:
df1

,0,1,2,3,4,5,6,7
None,百万円,％,百万円,％,百万円,％,百万円,％
2022年12月期第1四半期,32646,7.9,4845,3.3,5100,-1.8,3473,-4.3
2021年12月期第1四半期,30269,-5.1,4689,-3.8,5192,3.0,3628,-3.1
index,売上高,売上高,営業利益,営業利益,経常利益,経常利益,親会社株主に帰属する四半期純利益,親会社株主に帰属する四半期純利益


In [248]:
cn=1
num_cnt=0
for cnt,index in enumerate(index_list):
    if index!=[] and cn==1:
        new_df1=pd.DataFrame([number[k][1:] for k in range(num_cnt,len(index)+num_cnt)],index=index)
        num_cnt+=len(index)
        cn+=1
        continue
    elif index!=[] and cn==2:
        new_df2=pd.DataFrame([number[k][1:] for k in range(num_cnt,len(index)+num_cnt)],index=index)
        num_cnt+=len(index)
        #インデックスが同じなら合わせる
        if index==index_list[cnt-1]:
            new_df=pd.concat([new_df1,new_df2],axis=1)
        else:
            cn+=1

In [252]:
new_df

NameError: name 'new_df' is not defined

In [250]:
new_df2

,0,1,2,3,4,5,6,7,8
None,百万円,％,百万円,％,百万円,％,百万円,％,円銭
通期,162000,4.3,27000,3.6,28200,0.7,20200,2.5,259.85


In [168]:
str(type(df))=="<class 'NoneType'>"

True

In [157]:
type(df)

NoneType

In [165]:
NoPredict_text=check_text(text_list,non_text)
cnt=0
for i in NoPredict_text:
    if ("未定"in i)or ("控える" in i) or ("困難" in i)or("M&A" in i):
        cnt+=1
if cnt>0:
    print(True)

In [128]:
NoPredict_or(NoPredict_text)

True

In [296]:
import tqdm
li=[]
NoPreFlag=False
for i in tqdm.tqdm(range(0,len(non_list))):
    try:
        text=normalize_XBRL(non_list[i])
        text,pre_term = text_tag(text)
        text_list,non_text=create_text_list(text)
        NoPredict_text=check_text(text_list,non_text)
        table_list=create_table_list(text_list,col_dic)
        column,number=create_list(table_list)
        column_list,number = create_NoneAndCol(column,number)
        index_list=create_index(number)
        df1,df2,df3,df4=create_DataFrame(number,index_list)
        df=select_df(df2,df3,df4,pre_term)
        df1=create_df1_column(df1,column_list)
        df=create_df_column(df,column_list)

        if df==None:
            if NoPredict_or(NoPredict_text):
                NoPreFlag=True
    except:
        li.append(non_list[i])

100%|███████████████████████████████████████| 1154/1154 [00:33<00:00, 34.71it/s]


In [297]:
import tqdm
li=[]
NoPreFlag=False
for i in tqdm.tqdm(range(0,len(non_list))):
    df1,df2,pre_term,NoPredict_text,NoPreFlag,errFlag=main(non_list[i])
    if errFlag and not NoPreFlag:
        li.append(non_list[i])

100%|███████████████████████████████████████| 1154/1154 [00:32<00:00, 35.02it/s]


In [300]:
li

['xbrl_data/28010/20210421497149.htm',
 'xbrl_data/41830/20200406490183.htm',
 'xbrl_data/49670/20201014403631.htm',
 'xbrl_data/49670/20200716463792.htm',
 'xbrl_data/49670/20200417495299.htm',
 'xbrl_data/53430/20190507415202.htm',
 'xbrl_data/53430/20190123462529.htm',
 'xbrl_data/53430/20181002413621.htm',
 'xbrl_data/53430/20180726485386.htm',
 'xbrl_data/53430/20180302481598.htm',
 'xbrl_data/54010/20190205470139.htm',
 'xbrl_data/54010/20181030425598.htm',
 'xbrl_data/54010/20180706477808.htm',
 'xbrl_data/59330/20220428531754.htm',
 'xbrl_data/67150/20190614455003.htm',
 'xbrl_data/67150/20190419407282.htm',
 'xbrl_data/68610/20210401488727.htm',
 'xbrl_data/68610/20201203431321.htm',
 'xbrl_data/68610/20201016404744.htm',
 'xbrl_data/68610/20200710460826.htm',
 'xbrl_data/68610/20200408491301.htm',
 'xbrl_data/68610/20200121448865.htm',
 'xbrl_data/68610/20191021410174.htm',
 'xbrl_data/68610/20190719473572.htm',
 'xbrl_data/68610/20190418406690.htm',
 'xbrl_data/68610/2019012

In [309]:
dic={}
for i in range(0,len(li)):
     dic[li[i][10:15]]=1

In [311]:
len(dic.keys())

45

In [234]:
text_li=main(li[4])[3]

In [235]:
NoPredict_or(text_li)

False

In [236]:
text_li

['', '']

In [335]:
main(li[1])

(                0     1      2      3       4      5      6      7       0  \
 None          百万円     ％    百万円      ％     百万円      ％    百万円      ％      円銭   
 2020年3月期  1338987  -9.7  71636  -23.3   65517  -36.4  37944  -50.1  194.94   
 2019年3月期  1482909  11.6  93427   -9.7  102972   -6.6  76115    6.3  385.60   
 
            1     2    3    4  
 None      円銭     ％    ％    ％  
 2020年3月期   ―   7.0  4.4  5.4  
 2019年3月期   ―  14.3  7.0  6.3  ,
            0  1      2  3      4  5      6  7      8  9       10
 None      百万円  ％    百万円  ％    百万円  ％    百万円  ％    百万円  ％      円銭
 通期    1145000  ―  37000  ―  34000  ―  26000  ―  20000  ―  104.88,
 '2021年3月期の連結業績予想',
 ['\n3.2021年3月期の連結業績予想(2020年4月1日～2021年3月31日)当社は、2021年3月期第1四半期より国際財務報告基準(IFRS)を任意適用することを決定しており、以下の連結業績予想はIFRSベースで作成しております。そのため日本基準を適用している2020年3月期の実績値に対する増減率は記載しておりません。(％表示は、対前期増減率)table\n売上収益\n営業利益\n税引前利益\n当期利益\n親会社の所有者に帰属する当期利益\n基本的1株当たり当期利益trend\n百万円\n％\n百万円\n％\n百万円\n％\n百万円\n％\n百万円\n％\n円銭trend\n通期\n1145000\n―\n37000\n―\n34000\n―\n2

In [207]:
cnt=0
for i in NoPredict_text:
    if ("未定"in i)or ("控える" in i) or ("困難" in i)or("M&A" in i):
        cnt+=1

In [222]:
cnt=0
for i in text_li:
    if ("未定"in i)or ("控え" in i) or ("困難" in i)or("M&A" in i) or("コロナウイルス" in i):
        cnt+=1

In [224]:
cnt

2

['\n3.2022年3月期の連結業績予想(2021年4月1日～2022年3月31日)2020年5月12日付適時開示「国際財務報告基準(IFRS)の任意適用に関するお知らせ」の通り、2022年3月期の連結業績予想を含む2021年3月期の決算短信は、従来の日本基準に基づいて作成しております。連結業績予想は以下の通りとなります。通期売上高492500百万円、営業利益43900百万円、経常利益43000百万円、親会社株主に帰属する当期純利益28830百万円、1株当たり当期純利益150円18銭第2四半期(累計)売上高243900百万円、営業利益21800百万円、経常利益21200百万円、親会社株主に帰属する当期純利益15000百万円、1株当たり当期純利益78円14銭なお、当社は、2021年3月期の有価証券報告書における連結財務諸表からIFRSを任意適用し、IFRSに基づく2022年3月期の連結業績予想につきましては、明らかになり次第、公表いたします。trend\n\n※決算短信は公認会計士又は監査法人の監査の対象外です※業績予想の適切な利用に関する説明、その他特記事項(将来に関する記述等についてのご注意)本資料に記載されている業績見通し等の将来に関する記述は、当社が現在入手している情報及び合理的であると判断する一定の前提に基づいており、その達成を当社として約束する趣旨のものではありません。また、実際の業績等は様々な要因により大きく異なる可能性があります。業績予想の前提となる条件及び業績予想のご利用に当たっての注意事項等については、添付資料6ページ「1.経営成績等の概況(4)今後の見通し」をご覧ください。(決算補足説明資料及び決算説明会内容の入手方法)(1)決算補足説明資料は、ＴＤnetで本日開示するとともに、当社ウェブサイトにも掲載しております。(2)決算説明会資料は、当社ウェブサイトにも掲載しています。trend\n',
 'table\n3.2022年3月期の連結業績予想(2021年4月1日～2022年3月31日)2020年5月12日付適時開示「国際財務報告基準(IFRS)の任意適用に関するお知らせ」の通り、2022年3月期の連結業績予想を含む2021年3月期の決算短信は、従来の日本基準に基づいて作成しております。連結業績予想は以下の通りとなります。通期売上高49250

In [46]:
column.insert(len(column),"None")

In [47]:
column

['None',
 '売上高',
 '営業利益',
 '経常利益',
 '親会社株主に帰属する四半期純利益',
 '1株当たり四半期純利益',
 '潜在株式調整後1株当たり四半期純利益',
 '売上高',
 '営業利益',
 '経常利益',
 '親会社株主に帰属する当期純利益',
 'None',
 '1株当たり当期純利益',
 'None']